In [18]:
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 1. 데이터 
datasets = (load_iris())

x = pd.DataFrame(datasets["data"])
y = pd.DataFrame(datasets["target"])


In [19]:
# print(datasets.colums) 
print(x)    
print(y)
# print(x.describe)

       0    1    2    3
0    5.1  3.5  1.4  0.2
1    4.9  3.0  1.4  0.2
2    4.7  3.2  1.3  0.2
3    4.6  3.1  1.5  0.2
4    5.0  3.6  1.4  0.2
..   ...  ...  ...  ...
145  6.7  3.0  5.2  2.3
146  6.3  2.5  5.0  1.9
147  6.5  3.0  5.2  2.0
148  6.2  3.4  5.4  2.3
149  5.9  3.0  5.1  1.8

[150 rows x 4 columns]
     0
0    0
1    0
2    0
3    0
4    0
..  ..
145  2
146  2
147  2
148  2
149  2

[150 rows x 1 columns]


In [20]:
# print(x.info())
# print(x.describe())
# 데이터 분리
x_train, x_test, y_train,y_test = train_test_split(x,y,
                                                   train_size=0.8,
                                                   shuffle = True,
                                                #  stratify 는 데이터 불균형을 해결해줌
                                                   stratify=y
                                                #    random_state=21
                                                   )


# print(x.columns)
# y 의 첫번째 칼럼을 원핫 인코딩 변환
y_train[0] = tf.one_hot(y_train[0],3)
y_test[0] = tf.one_hot(y_test[0],3)

y_train.shape


(120, 1)

In [21]:
model = Sequential([
    Dense(50,activation="relu",input_shape=(4,)),
    Dense(40,activation="relu") ,
    Dense(40,activation="relu") ,
    Dense(30,activation="relu") ,
    Dense(30,activation="relu") ,
    Dense(20,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(3,activation="softmax")
])

In [22]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
min_delta=0.01, patience=5, 
verbose=1, 
mode='min')

# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=100, batch_size=32, 
                 validation_split=0.2,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


Epoch 1/100
3/3 [==============================] - 1s 119ms/step - loss: 1.1336 - accuracy: 0.1979 - val_loss: 1.0637 - val_accuracy: 0.4167
Epoch 2/100
3/3 [==============================] - 0s 12ms/step - loss: 1.0558 - accuracy: 0.7396 - val_loss: 1.0221 - val_accuracy: 0.7917
Epoch 3/100
3/3 [==============================] - 0s 13ms/step - loss: 1.0175 - accuracy: 0.7708 - val_loss: 0.9800 - val_accuracy: 0.6667
Epoch 4/100
3/3 [==============================] - 0s 13ms/step - loss: 0.9813 - accuracy: 0.6667 - val_loss: 0.9451 - val_accuracy: 0.6667
Epoch 5/100
3/3 [==============================] - 0s 14ms/step - loss: 0.9480 - accuracy: 0.6979 - val_loss: 0.9163 - val_accuracy: 0.8750
Epoch 6/100
3/3 [==============================] - 0s 13ms/step - loss: 0.9147 - accuracy: 0.8125 - val_loss: 0.8921 - val_accuracy: 0.5833
Epoch 7/100
3/3 [==============================] - 0s 13ms/step - loss: 0.8833 - accuracy: 0.6875 - val_loss: 0.8610 - val_accuracy: 0.5833
Epoch 8/100
3/3 [==

In [23]:
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


1/1 [==============================] - 0s 18ms/step - loss: 0.1980 - accuracy: 0.9333
loss: 0.19799955189228058 
acc : 0.9333333373069763


In [24]:
from sklearn.metrics import accuracy_score
y_predict = model.predict(x_test)

# predict 의 값은 소숫점값이라 정수형으로 바꿔줘야합니다.
print(y_predict[:5])

# argmax 는 x 를 최대값으로 만들어주는 입력을 구하는함수
y_predict = np.argmax(y_predict,axis=1)

print(y_predict[:5])
# 원핫 인코딩 변환
y_predict = tf.one_hot(y_predict,3)

print(y_predict[:5])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print(acc)

[[9.4865100e-05 6.9023441e-03 9.9300283e-01]
 [8.0395979e-04 9.2166322e-01 7.7532865e-02]
 [2.1605556e-05 2.0101790e-03 9.9796814e-01]
 [6.8326847e-04 1.9940288e-01 7.9991388e-01]
 [9.9879754e-01 1.1586258e-03 4.3868265e-05]]
[2 1 2 2 0]
tf.Tensor(
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]], shape=(5, 3), dtype=float32)
0.9333333333333333
